# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
!pip uninstall keras
!pip install tf-keras==2.16.0 tensorflow-hub==0.16.0

In [ ]:
import numpy as np
from game_2048 import Game2048, choose_action_heuristic
import keras
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
env = Game2048()

states = []
actions = []

NUM_EPISODES = 300   # pode aumentar depois
MAX_MOVES    = 300

for ep in range(NUM_EPISODES):
    env.reset()
    done = False
    moves = 0

    while not done and moves < MAX_MOVES:
        state_vec = Game2048.encode_board(env.board)
        action = choose_action_heuristic(env)

        states.append(state_vec)
        actions.append(action)

        _, _, done = env.step(action)
        moves += 1

len(states), len(actions)


In [ ]:
X = np.stack(states, axis=0)               # shape (N,16)
y = np.array(actions, dtype=np.int64)      # shape (N,)

X.shape, y.shape


In [ ]:
NUM_ACTIONS = 4
STATE_SIZE = 16

model = keras.models.Sequential([
    layers.Input(shape=(STATE_SIZE,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_ACTIONS, activation='softmax')  # classificação de ação
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X, y,
    batch_size=64,
    epochs=20,
    validation_split=0.1,
    verbose=1
)


In [ ]:
# Monkey patch pros aliases que o tensorflowjs espera
np.bool = bool
np.object = object

import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "../model")
